In [1]:
from test_structure import *
from GPyOpt.methods import BayesianOptimization
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(1)

In [17]:
def beam_n(X, n, EI_req = 1*10**6):
    """
    beam model
    function for n extra non basic nodes
    X[0:3*n] are node locations (x, y, z for each node)
    X[3*n:] are member diameters (8+n*8+n*(n-1)/2)
    EI_req is required EI
    """
    beam = test_struct()
    node_locs = np.array(X[0:3*n]).reshape(n, 3)
    beam.make_nodes(node_locs)
    beam.make_mem_ps(X[3*n:])
    beam.make_struct()
    beam.record_struct_info()
    EI = beam.get_EI(100)
    print(EI)
    
    if np.isnan(EI) or beam.min_l<0.01:
        return(10**9)
    else:
        return(beam.mass+10**7*max(1/EI-1/EI_req, 0))

In [18]:
def truss_n(X, n, EI_req = 1*10**6):
    """
    truss model (release moments at nodes)
    function for n extra non basic nodes
    X[0:3*n] are node locations (x, y, z for each node)
    X[3*n:] are member diameters (8+n*8+n*(n-1)/2)
    EI_req is required EI
    """
    truss = test_struct()
    node_locs = np.array(X[0:3*n]).reshape(n, 3)
    truss.make_nodes(node_locs)
    truss.make_mem_ps(X[3*n:])
    truss.make_struct()
    truss.release_moments()
    truss.record_struct_info()
    EI = truss.get_EI(100)
    print(EI)
    
    if np.isnan(EI) or truss.min_l<0.01:
        return(10**9)
    else:
        return(truss.mass+10**7*max(1/EI-1/EI_req, 0))

In [24]:
def unit_n(X, n, EI_req = 1*10**6):
    """
    unit cell beam model
    function for n extra non basic nodes
    X[0:3*n] are node locations (x, y, z for each node)
    X[3*n:] are member diameters (8+n*8+n*(n-1)/2)
    EI_req is required EI
    """
    
    beam = test_struct(num_units=1)
    node_locs = np.array(X[0:3*n]).reshape(n, 3)
    beam.make_nodes(node_locs)
    beam.make_mem_ps(X[3*n:])
    beam.make_struct()
    beam.record_struct_info()
    EI = beam.get_EI(100)
    print(EI)

    if np.isnan(EI) or beam.min_l<0.01:
        return(10**9)
    else:
        return(beam.mass+10**7*max(1/EI-1/EI_req, 0))

In [20]:
def equiv_cant_n(X, n, EI_req = 1*10**6):
    """
    EI calculated using equivalent cantilever model (circular crosssection, match mass)
    function for n extra non basic nodes
    X[0:3*n] are node locations (x, y, z for each node)
    X[3*n:] are member diameters (8+n*8+n*(n-1)/2)
    EI_req is required EI
    """
    beam = test_struct(num_units=1)
    node_locs = np.array(X[0:3*n]).reshape(n, 3)
    beam.make_nodes(node_locs)
    beam.make_mem_ps(X[3*n:])
    beam.make_struct()
    beam.record_struct_info()
    equiv_EI = beam.get_equiv_EI()
    print(equiv_EI)
    
    if np.isnan(equiv_EI) or beam.min_l<0.01:
        return(10**9)
    else:
        return(beam.mass+10**7*max(1/equiv_EI-1/EI_req, 0))

In [21]:
#test nodes
node_locs = [0.5, 0.2, 0.3, 0.6, 0.8, 1, 0, 0.2, 0.5]
member_ds = [0.004]*35
X = [node_locs+member_ds]

In [25]:
n = 3
hf_f = beam_n
lf_f = unit_n
def f_hf(X):
    return(hf_f(X[0], n, 1*10**6))
def f_lf(X):
    return(lf_f(X[0], n, 1*10**6))

In [ ]:
node_domain = [{'name':'node_coord', 'type':'continuous', 'domain':(0, 1)}]
d_domain = [{'name':'member_d', 'type':'continuous', 'domain':(0, 0.1)}]

domain = node_domain*n*3 + d_domain*int((n+1)*8+n*(n-1)/2)
#optimising for 1 non basic node, require EI 1*10**6
myOpt = BayesianOptimization(f_hf,
                             domain=domain,
                             acquisition_type="EI",
                             model_type='GP',
                             exact_feval=True)
myOpt.run_optimization(max_iter = 100, max_time = 600, eps=1e-6)

In [ ]:
myOpt.plot_convergence()

In [ ]:
print(myOpt.x_opt)

In [ ]:
opt_beam, EI, mass, min_l= do_beam(myOpt.x_opt, n)
print(EI)
print(mass)
print(min_l)

In [ ]:
Visualization.RenderModel(opt_beam.struct, deformed_shape=True, deformed_scale=1, text_height=0.05)